In [2]:
import GCRCatalogs
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf

In [6]:
# read in redmagic catalog
redmagic = pf.open('/global/projecta/projectdirs/lsst/groups/WL/users/zuntz/data/redmagic/cosmoDC2_v1.1.4_run_redmagic_highdens.fit')
redmagic_ids = redmagic[1].data['id']

In [8]:
gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

In [17]:
# GCRCatalogs.get_available_catalogs(names_only=True)

In [18]:
halo_all = gc.get_quantities(['halo_mass', 'mag_i', 'redshift', 'halo_id'], filters=['isCentral', 'redshift < 1.5'])

In [ ]:
from scipy.stats import binned_statistic, binned_statistic_2d